### Exploratory Analysis

This notebook was created to explore the Admissions dataset for the first time.  The goal was to create as many visualizations as possible in order to gain an understanding of the dataset.  Some of the code used to create visualizations can be found [here](https://github.com/stibbs1998/admissions_internship/blob/master/src/visualization/visualize.py).

Import necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import warnings
import folium
import sys
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src/visualization/')
import visualize as vis

import altair as alt
# alt.renderers.enable('notebook')

Load in the .csv file as a DataFrame.  The original file was cleaned up using [this code](https://github.com/stibbs1998/admissions_internship/blob/master/src/data/001-st-clean_data.py), and in this notebook we are using the dataset cleaned only to the first order.  This meant removing Albany Med and HEOP applicants, as well as converting the *Enrolled* column from a list of type strings to type booleans.

In [ ]:
filename = '../data/processed/CriticalPath_Data_EM_Confidential_lessNoise.csv'
df = pd.read_csv(filename).drop(columns=["Unnamed: 0"])

Create a Histogram and [Kernel Density Estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation) (or KDE) of each student's High School GPA.

* All applicants

In [ ]:
all_hs_gpa = df[~df['HS_GPA'].isnull()][(df['HS_GPA']>5)& (df['HS_GPA']<400)]

# There are a couple of errors in the data set for students whose GPA's were sent in on a 0-4 scale
# instead of a 0-100 scale, and a few GPA's greater than 500.  For the purpose of this notebook, we will
# just ignore these points.

vis.kde_w_mean(all_hs_gpa['HS_GPA'],20, "# of Students")

plt.xlabel("High School GPA (out of 100)")
plt.title("KDE of High School GPA: All Applicants");

* Students Enrolled

In [ ]:
vis.kde_w_mean(all_hs_gpa['HS_GPA'][all_hs_gpa['Enrolled']],20, "# of Students")

plt.xlabel("High School GPA (out of 100)")
plt.title("KDE of High School GPA: Enrolled");

Create a Histogram and KDE of each student's High School Percentile Ranking.


* All applicants

In [ ]:
all_hs_percentile = df[~df['HS_Percentile_rank'].isnull()][df['HS_Percentile_rank']>=0]
vis.kde_w_mean(all_hs_percentile['HS_Percentile_rank'],100, "# of Students")
plt.xlabel("Percentile Rank in HS Class")
plt.title("Distribution of HS Percentile Rankings: All Applicants");

* Enrolled

In [ ]:
vis.kde_w_mean(all_hs_percentile['HS_Percentile_rank'][all_hs_percentile['Enrolled']],100, "# of Students")
plt.xlabel("Percentile Rank in HS Class")
plt.title("Distribution of HS Percentile Rankings: Enrolled");

Create a Histogram and KDE of each student's High School Class Size.

* All Applicants

In [ ]:
hs_size = df[~df['HS_Class_size'].isnull()]
vis.kde_w_mean(hs_size['HS_Class_size'],100, "# of Students")
plt.title("Distribution of Class Sizes: All Applicants");
plt.xlim(0,1400);

* Enrolled Students

In [ ]:
hs_size = df[~df['HS_Class_size'].isnull()]
vis.kde_w_mean(hs_size['HS_Class_size'][hs_size['Admission_status']==80],100, "# of Students")
plt.ylabel("Probability Density Function")
plt.title("Distribution of Class Sizes: Enrolled");
plt.xlim(0,1400);

Create a Histogram and KDE of each student's ACT Scores

* All Applicants

In [ ]:
act_scores = df[~df['ACTComposite'].isnull()]
vis.kde_w_mean(act_scores['ACTComposite'],25,'# of Students')
plt.title("Distribution of ACT Scores: All Applicants");

* Enrolled Students

In [ ]:
vis.kde_w_mean(act_scores['ACTComposite'][act_scores['Enrolled']],20,'# of Students')
plt.title("Distribution of ACT Scores: Enrolled");

Create a Histogram and KDE of each student's SAT Scores

* New SAT Format

$\to$ All Applicants

In [ ]:
for column in df.columns:
    if "NEW" in column:
        plt.figure()
        vis.kde_w_mean(df[column][~df[column].isnull()], 20, "# of Students")
        plt.title(f"Distribution of {' '.join(column.split('_')) }: All Applicants")

$\to$ Enrolled Students

In [ ]:
for column in df.columns:
    if "NEW" in column:
        plt.figure()
        vis.kde_w_mean(df[column][(~df[column].isnull()) & (df['Enrolled'])], 20, "# of Students")
        plt.title(f"Distribution of {' '.join(column.split('_')) }: Enrolled")

* Old SAT Format

$\to$ All Applicants

In [ ]:
for column in df.columns:
    if "NEW" not in column and "SAT" in column:
        plt.figure()
        vis.kde_w_mean(df[column][~df[column].isnull()], 20, "# of Students")
        plt.title(f"Distribution of {' '.join(column.split('_')) }: All Applicants")

$\to$ Enrolled Students

In [ ]:
for column in df.columns:
    if "NEW" not in column and "SAT" in column:
        plt.figure()
        vis.kde_w_mean(df[column][(~df[column].isnull()) & df['Enrolled']], 20, "# of Students")
        plt.title(f"Distribution of {' '.join(column.split('_')) }: Enrolled")

Breakdown of Applications by Major on application. 

First, create a subset of the DataFrame to pull information from.

In [ ]:
num_majors_not_enrolled = []
num_majors_enrolled = []

for major in df['Major'].unique():
    num_majors_not_enrolled.append(df.Major.str.count(major).sum())
    num_majors_enrolled.append(df.Major[df['Enrolled']].str.count(major).sum())

majorsDF = pd.DataFrame({
    "Major": df.Major.unique(), 
    "Enrolled": num_majors_enrolled,
    "Applied":  num_majors_not_enrolled})

majorsDF['Percentage Enrolled'] = majorsDF['Enrolled'] / majorsDF['Applied']

Use [Altair](https://altair-viz.github.io/) to create bar charts for number of students applied and students enrolled by major.

In [ ]:
enrolled_top15 = alt.Chart(majorsDF[:15]).mark_bar(color='gold').encode(
    x=alt.X('Enrolled:Q',axis=alt.Axis(title='# Students Enrolled')),
    y=alt.Y(
        "Major:O",
        sort=alt.EncodingSortField(
          field = 'Enrolled',
          op = 'sum',
          order = 'descending'
        )
    )
).properties(height=200,width=300,title='Enrollment by Major')

applied_top15 = alt.Chart(majorsDF[:15]).mark_bar(color='green').encode(
    x=alt.X('Applied:Q',axis=alt.Axis(title='# Students Applied')),
    y=alt.Y(
        "Major:O",
        sort=alt.EncodingSortField(
          field = 'Enrolled',
          op = 'sum',
          order = 'descending'
        )
    )
).properties(height=200,width=300,title="Applications by Major")

percent_top15 = alt.Chart(majorsDF[:15]).mark_bar().encode(
    x=alt.X('Percentage Enrolled:Q',axis=alt.Axis(format='%')),
    y=alt.Y(
        "Major:O",
        sort=alt.EncodingSortField(
            field="Applied",
            op="sum",
            order="descending"
        )
    )
).properties(height=200,width=300,title="Enrollment:Application Percentage")

(percent_top15 & (applied_top15|enrolled_top15))

Breakdown of Non-NYS applicants by State

First, create a DataFrame to pull information from.

In [ ]:
state_counts = []
states = []
athletes = []

for state in df['State_perm_res'][df['International_student']=='US'].unique():
    
    if type(state)==str and state!='NY':
        states.append(state)  
        state_counts.append(df['State_perm_res'].str.count(state).sum())
        athletes.append(
            df['State_perm_res'][~df['Recruited_athlete'].isnull()].str.count(state).sum()
        )
    
statesDF = pd.DataFrame({"State": states, "# Students":state_counts,
                         "# Athletes": athletes})

statesDF = statesDF.sort_values("# Students",ascending = False).reset_index()
statesDF['Non-Athletes'] = statesDF['# Students'] - statesDF['# Athletes']

Bar graph of the top 10 states by applications outside of NY.

In [ ]:
alt.Chart(statesDF.iloc[:10]).mark_bar().encode(
    x='# Students:Q',
    y=alt.Y(
        "State:O", axis=alt.Axis(title=''),
        sort=alt.EncodingSortField(
            field="# Students",
            op="sum",
            order="descending"
        )
    )
).properties(height=300,title='Applications by State')

### [Choropleth Maps](https://en.wikipedia.org/wiki/Choropleth_map)

First, a map of applicants by state (non-NYS residents).

In [ ]:
usa = '../data/processed/us-states.json'

m = folium.Map(
    location=[42.65, -73.75],
    tiles='Mapbox Bright',
    zoom_start=5
)

folium.Choropleth(
    geo_data=usa,
    nan_fill_opacity=0,
    name='choropleth',
    data=statesDF,
    columns=['State', '# Students'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Admission by State',
    highlight=True
).add_to(m)

folium.LayerControl().add_to(m)

m

Since the majority of out-of-state applicants come from three states, suppress these states from the data as well.

In [ ]:
m = folium.Map(
    location=[42.65, -73.75],
    tiles='Mapbox Bright',
    zoom_start=5
)

folium.Choropleth(
    geo_data=usa,
    nan_fill_opacity=0,
    name='choropleth',
    data=statesDF.sort_values("# Students",ascending=False).iloc[3:],
    columns=['State', '# Students'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Applications by State',
    highlight=True
).add_to(m)

folium.LayerControl().add_to(m)

m

Does the number of athletes recruited from out of state impact the data noticably?

From the looks of this graph, there is little difference in popularity of nearby states, but not by a substantial margin.

In [ ]:
m = folium.Map(
    location=[42.65, -73.75],
    tiles='Mapbox Bright',
    zoom_start=5
)

folium.Choropleth(
    geo_data=usa,
    nan_fill_opacity=0,
    name='choropleth',
    data=statesDF.sort_values("# Students",ascending=False).iloc[3:],
    columns=['State', 'Non-Athletes'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Admission by State',
    highlight=True
).add_to(m)

folium.LayerControl().add_to(m)

m

Breakdown of applicants by city of residence.

First, create a DataFrame to pull information from.

In [ ]:
citiesDF = df[df['Enrolled']].groupby(["City_perm_res",
  "State_perm_res"]).count().sort_values("Unique_student_ID",ascending=False)[
    ['Unique_student_ID']].rename(columns = {"Unique_student_ID":"Enrolled"})

citiesDF = citiesDF.merge(
    df.groupby(["City_perm_res","State_perm_res"]).count().sort_values("Unique_student_ID",
    ascending=False)[['Unique_student_ID']].rename(columns = {"Unique_student_ID":"Applied"}),
    left_index=True, right_index=True)

citiesDF['Percent Enrolled'] = citiesDF['Enrolled']/citiesDF['Applied']

citiesDF.reset_index(inplace=True)
citiesDF = citiesDF.sort_values("Enrolled",ascending=False)

Barplot of the top ten cities by enrollment total.

The top ten cities are **dominated** by the Capital Region.

In [ ]:
alt.Chart(citiesDF.iloc[:10]).mark_bar().encode(
    x=alt.X('Enrolled:Q', axis=alt.Axis(title='# Students Enrolled')),
    y=alt.Y(
        'City_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=250,title="Enrollment Totals by City")

Barplot of the top ten cities *outside* of NYS.

In [ ]:
alt.Chart(citiesDF[citiesDF["State_perm_res"]!='NY'][:10]).mark_bar().encode(
    x=alt.X('Enrolled:Q',axis=alt.Axis(title='# Students Enrolled')),
    y=alt.Y('City_perm_res:O',axis=alt.Axis(title=''),
            sort = alt.EncodingSortField(
                field='Enrolled',
                op = "sum",
                order = "descending"
            )
        )
).properties(height=250,title='Enrollment by City (Non-NYS Residents)')

What are the top 10 and bottom 10 cities by enrollment yield?

(Assuming a minimum of 20 applicants from that city)

In [ ]:
bottom10cities = alt.Chart(citiesDF[citiesDF['Applied']>20].sort_values("Percent Enrolled").iloc[:10]).mark_bar().encode(
    x=alt.X('Percent Enrolled:Q', axis=alt.Axis(format='%')),
    y=alt.Y(
        'City_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Percent Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=200,width=200,title="Worst Cities by Enrollment Yield")

top10cities = alt.Chart(citiesDF[citiesDF['Applied']>20].sort_values("Percent Enrolled",ascending=False).iloc[:10]).mark_bar().encode(
    x=alt.X('Percent Enrolled:Q', axis=alt.Axis(format='%')),
    y=alt.Y(
        'City_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Percent Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=200,width=200,title="Best Cities by Enrollment Yield")

bottom10cities | top10cities

Scatter plot of # of applications vs. # of students enrolled by city.

In [ ]:
f, axes = plt.subplots(figsize=(10,6))
sns.scatterplot(data = citiesDF, x="Applied", y="Enrolled");
plt.title("Applied v. Attended");

Breakdown of applicants by county of residence.

First, create a DataFrame to pull information from.

In [ ]:
countiesDF = df[df['Enrolled']].groupby(["County_perm_res",
  "State_perm_res"]).count().sort_values("Unique_student_ID",
  ascending=False)[['Unique_student_ID']].rename(columns = {"Unique_student_ID":"Enrolled"})

countiesDF = countiesDF.merge(
    df.groupby(["County_perm_res","State_perm_res"]).count().sort_values("Unique_student_ID",
    ascending=False)[['Unique_student_ID']].rename(columns = {"Unique_student_ID":"Applied"}),
    left_index=True, right_index=True)

countiesDF['Percent Enrolled'] = countiesDF['Enrolled']/countiesDF['Applied']
countiesDF.reset_index(inplace=True)
countiesDF = countiesDF.sort_values("Enrolled",ascending=False)

######################
######################
## convert county codes to names 

fips_codes = pd.read_csv('../data/processed/FIPS.csv')
fips_codes.State + fips_codes.FIPS.astype(str)
fips_codes['FIPS'] = (fips_codes['FIPS'] - np.floor(fips_codes['FIPS']/1000)*1000).astype(int).astype(str)
fips_codes['FIPS'] = fips_codes['FIPS'].apply(lambda x: x.zfill(3))
fips_codes['FIPS'] = fips_codes.State + fips_codes.FIPS

mapper = fips_codes.set_index('FIPS')[['Name']].to_dict()['Name']
countiesDF['County_perm_res'] = countiesDF['County_perm_res'].map(mapper)

Barplot of the top counties by enrollment.

In [ ]:
alt.Chart(countiesDF.iloc[:10]).mark_bar().encode(
    x=alt.X('Enrolled:Q', axis=alt.Axis(title='# Students Enrolled')),
    y=alt.Y(
        'County_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=250,title="Enrollment Totals by County")

Once again, this seems to be dominated by NYS residents, so lets supress NY and take a look at the best counties out of state.

In [ ]:
alt.Chart(countiesDF[countiesDF['State_perm_res']!='NY'].iloc[:10]).mark_bar().encode(
    x=alt.X('Enrolled:Q', axis=alt.Axis(title='# Students Enrolled')),
    y=alt.Y(
        'County_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=250,title="Enrollment Totals by County")

What are the top 10 and bottom 10 counties by enrollment yield?

(Assuming a minimum of 20 applicants from that county)

In [ ]:
bottom10counties = alt.Chart(countiesDF[countiesDF['Applied']>20].sort_values("Percent Enrolled").iloc[:10]).mark_bar(color='green').encode(
    x=alt.X('Percent Enrolled:Q', axis=alt.Axis(format='%')),
    y=alt.Y(
        'County_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Percent Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=200,width=200,title="Worst Counties by Enrollment Yield")

top10counties = alt.Chart(countiesDF[countiesDF['Applied']>20].sort_values("Percent Enrolled",ascending=False).iloc[:10]).mark_bar(color='gold').encode(
    x=alt.X('Percent Enrolled:Q', axis=alt.Axis(format='%')),
    y=alt.Y(
        'County_perm_res:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='Percent Enrolled',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=200,width=200,title="Best Counties by Enrollment Yield")

bottom10counties | top10counties

Scatterplot of # of applications vs. # of students enrolled by county.

In [ ]:
f, axes = plt.subplots(figsize=(10,6))
sns.scatterplot(data = countiesDF, x="Applied", y="Enrolled");
plt.title("Applied v. Attended");

Create a DataFrame to analyze parents' financial status by race/ethnicity.

In [ ]:
ethnicity_AGIDF = pd.merge(left=df[['Ethnicity','Parent_income_AGI']].groupby("Ethnicity").mean(),
  right=df[["Ethnicity","Enrolled","Unique_student_ID"]].groupby("Ethnicity").count(),
          how='outer', left_index=True, right_index=True).rename(columns={"Unique_student_ID":"TotApplicants",
                                                                         "Parent_income_AGI":"AvgParent_income_AGI"})
ethnicity_AGIDF.reset_index(inplace=True)
ethnicity_AGIDF = ethnicity_AGIDF.drop(7)

Barplot of Parent's average adjusted gross income.

In [ ]:
alt.Chart(ethnicity_AGIDF).mark_bar().encode(
    x=alt.X('AvgParent_income_AGI:Q', axis=alt.Axis(format='$,k',title="Parents' AGI",labelAngle=20)),
    y=alt.Y(
        'Ethnicity:O', axis=alt.Axis(title=''),
        sort = alt.EncodingSortField(
                field='AvgParent_income_AGI',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=200,width=300,title='Average AGI by Race/Ethnicity')

Boxplot of parents AGI by race/ethnicity.

In [ ]:
import matplotlib.ticker as mtick

f, axes = plt.subplots(figsize=(10,7))
sns.boxplot(data=df[df['Ethnicity']!='Unknown'],y="Ethnicity",x="Parent_income_AGI")

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

axes.xaxis.set_major_formatter(tick) 

plt.xticks(rotation=-20)
plt.xlim(0,500000);